# Introduction

The United States House of Representatives is the lower chamber of the United States Congress, the Senate being the upper chamber. Together they comprise the legislature of the United States.[1] Congressional districts in the United States are electoral divisions for the purpose of electing members of the United States House of Representatives. The number of voting seats in the House of Representatives is currently set at 435 with each one representing approximately 711,000 people.[2]  
The One Hundred Fifteenth United States Congress is the current meeting of the legislative branch of the United States federal government, composed of the Senate and the House of Representatives. It meets in Washington, D.C. from January 3, 2017, to January 3, 2019, during the final weeks of Barack Obama's presidency and the first two years of Donald Trump's presidency.[3]
The Republican Party, also referred to as the GOP (Grand Old Party), is one of the two major political parties in the United States, the other being its historic rival, the Democratic Party. Currently, their ideology is American conservatism, which contrasts with the Democrats' liberal platform and progressive wing. The GOP's political platform supports lower taxes, free market capitalism, free enterprise, a strong national defense, gun rights, deregulation and restrictions on labor unions. In addition to advocating for conservative economic policies, the Republican Party is socially conservative and seeks to uphold traditional values based largely on Judeo-Christian ethics.[4]  
The Democrats' dominant worldview was once social conservatism and economic liberalism while populism was its leading characteristic in the rural South.Today, the House Democratic caucus is composed mostly of centrists and progressives, with a small minority of conservative Democrats. The party's philosophy of modern liberalism advocates social and economic equality, along with the welfare state. It seeks to provide government intervention and regulation in the economy. These interventions, such as the introduction of social programs, support for labor unions, affordable college tuitions, moves toward universal health care and equal opportunity, consumer protection and environmental protection form the core of the party's economic policy.[5]


1.[United States House of Representatives.wikipedia](https://en.wikipedia.org/wiki/United_States_House_of_Representatives)  
2.[List of United States congressional districts.wikipedia](https://en.wikipedia.org/wiki/List_of_United_States_congressional_districts)  
3.[115th United States Congress.wikipedia](https://en.wikipedia.org/wiki/115th_United_States_Congress)  
4.[Republican Party (United States).wikipedia](https://en.wikipedia.org/wiki/Republican_Party_(United_States))  
5.[Democratic Party (United States).wikipedia](https://en.wikipedia.org/wiki/Democratic_Party_(United_States))



# Data

The data file party115cong.csv contains data on all congressional districts1 represented in the U.S. House of Representatives during the 115th U.S. Congress. Each row represents a district, and the columns are as follows:  
* state: the U.S. state containing the district, or the District of Columbia
* district: an identifier of Congressional district within state
* electedrep: name of person elected to the 115th Congress
* party: the party affiliation of the Representative during the 115th Congress:D for Democrat, R for Republican. In this data, 115th U.S. Congress includes 195 Democrat representatives and 241 Republican representatives.
* medHouseIncome: median household income (dollars), a histogram of this variable is showing below:
![median household income](medHouseIncome.jpg)

# First Model

Fit a Bayesian logistic regression model to explain party based on the natural logarithm of medHouseIncome, The response will be Bernoulli: 1 if Democrat (D), 0 if Republican (R). The model will be a simple model with an ordinary “intercept” term and a coefficient multiplying the (centered and rescaled to have sample standard deviation of 0.5) log(medHouseIncome).  
**(a)A JAGS model is listed below:**

In [ ]:
#JAGS model
#party.bug
'
model {
  for (i in 1:length(party)) {
    party[i] ~ dbern(prob[i])
    logit(prob[i]) <- betaincome*incomescaled[i]+intercept
    partyrep[i] ~ dbern(prob[i])

  }
  betaincome ~ dt(0, 0.16, 1)
  intercept ~ dt(0, 0.01, 1)
}
'

**(b)Computation Summary:**  
4 chains are used, with 1000 adaptation and 1000 iterations for burn-in, convergence is reached after 2000 iteration. after 4000 sampling for each chain, the effective sample size for **betaincome** is 10444, effective sample size for **intercept** is 10009.  

**(c)Approximate posterior parameter:**  

**betaincome**  
* mean: 0.3588
* posterior standard deviation: 0.1938
* 95% central posterior interval: (-0.01728,0.74050)

**intercept**:
* mean: -0.2147
* posterior standard deviation: 0.0971
* 95% central posterior interval: (-0.40412,-0.02608)

**(d)Approximate the posterior probability that the “slope” exceeds zero:**  
The result shows that the posterior probability of “slope” exceeds zero is 0.970, which can be interpreted as statistical significance. The model indicates that median household income has positive impact on electing a Democrat.

**(e)DIC and the associated effective number of parameters:**  
The Penalized deviance is 600 for this model, and effective number of parameters is 1.966, whereas the acutally number of parameter is 2.

# Scond Model

Extend the first model by allowing each state to have a separate additive random effect based on state: add to the linear portion of the model a random effect term that varies by state. the variable state is recoded with the integers 1 to 51. Let the prior for these random effects be (conditionally) independent from a normal distribution with mean zero (since the model already has an intercept) and standard deviation σstate. Let the prior for σstate be approximately flat.  
**(a)A JAGS model is listed below:**

In [ ]:
#JAGS model
#party2.bug
'
model {
  for (i in 1:length(party)) {
    party[i] ~ dbern(prob[i])
    logit(prob[i]) <- betaincome*incomescaled[i]+betastate[state[i]]+intercept
    partyrep[i] ~ dbern(prob[i])

  }
  for (j in 1:max(state)) {
    betastate[j] ~ dnorm(0, 1/sigmastate^2)
  }
  betaincome ~ dt(0, 0.16, 1)
  intercept ~ dt(0, 0.01, 1)
  sigmastateinv ~ dgamma(10, 10)
  sigmastate <- inverse(sigmastateinv)
}
'

**(b)Computation Summary:**  
4 chains are used, with 1000 adaptation and 10000 iterations for burn-in, convergence is reached after 2000 iteration. after 8000 sampling for each chain, the effective sample size for **betaincome** is 5531, effective sample size for **intercept** is 2204.  

**(c)Approximate posterior parameter:**   

**betaincome**  
* mean: -0.470867
* posterior standard deviation: 0.264880
* 95% central posterior interval: (-0.99562, 0.04125)

**intercept**:
* mean: -0.458587
* posterior standard deviation: 0.231352
* 95% central posterior interval: (-0.93163, -0.02433)  

**(d)Approximate the posterior probability that the “slope” exceeds zero:**  
The result shows that the posterior probability of “slope” exceeds zero is 0.036, which can be interpreted as statistical significance. The model indicates that median household income has negative impact on electing a Democrat,  after adjustment for state, suggests that median income can be used for predicting between Democrat and Republican at the country level, but not in the state level.  

**(e)posterior mean random effect:**  
Based on the results in appendix, Massachusetts has the largest (in the positive direction) posterior mean random effect, whereas Oklahoma has the smallest (in the negative direction) posterior mean random effect. This result matches what we know about the US politics, Massachusetts has more diverse population and elite eduacation which advocates social and economic equality, so that higher house income may indicates better education of that family, so they tend to agree with Democrat philosophy, therefore house income shows positive predictive impact on electing Democrat.  

**(f)DIC and the associated effective number of parameters:**  
The Penalized deviance is 541.8 for this model, and effective number of parameters is 31.23, whereas the acutally number of parameter is 53.  
In my opinion, the both models are similarily good, their Penalized deviance is not very different, although the second model has more number of parameters, it gives more percise insight about the election for each state. Overall, the two models provided different angles for the prediction.


# Conclusions

1. The distribution of median house income is approximately a skewed normal distribution.
2. Using the bayesian logistic model that only takes median house income in consideration, we can predict that the higher income favors Democrat.
3. Using the bayesian logistic model that takes median house income as variable but add state as an adjustment, we can see that higher income is not alway associated with electing Democrat, because state has a strong effect, and this matches our background knowledge about US politics.

# Appendix
R code for the computation is in below with comments

In [70]:
#read data table
Data <- read.csv(file = 'party115cong.csv',header = TRUE)

In [71]:
head(Data)

state,district,electedrep,party,medHouseIncome
Alabama,1,Bradley Byrne,R,47083
Alabama,2,Martha Roby,R,42035
Alabama,3,Mike Rogers,R,46544
Alabama,4,Robert Aderholt,R,41110
Alabama,5,Mo Brooks,R,51690
Alabama,6,Gary Palmer,R,61413


In [3]:
#overall count of representative by party 
library(plyr)
count(Data, "party")

party,freq
D,195
R,241


In [10]:
#plot the medHouseIncome and save to file
jpeg("medHouseIncome.jpg", width = 500, height = 500)
hist(Data$medHouseIncome)
dev.off()

pdf 
  2

In [11]:
###code for First Model listing in below

In [72]:
#create a variable to encode the party category
unclass(Data$party)
Data$Bparty<-as.numeric(Data$party)
Data$Bparty[Data$Bparty==2]<-0
Data$Bparty[Data$Bparty==1]<-1
#Data$Bparty<-as.factor(Data$Bparty)

[1] 2 2 2 2 2 2 1 2 1 2 1 2 2 2 1 2 1 2 2 2 2 2 1 1 2 1 1 1 2 1 2 1 1 1 1 1 1
 [38] 1 1 1 1 2 2 2 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 1 2 1 1 2 1 1 2 2 2 1 1 1
 [75] 1 1 2 2 2 2 1 1 1 1 1 1 1 1 2 2 2 2 1 2 1 2 1 1 2 2 1 1 2 2 2 2 2 1 1 1 1
[112] 1 2 2 2 2 1 2 1 1 2 2 2 2 2 2 2 1 2 1 1 2 2 1 1 1 1 1 2 1 1 1 1 1 2 2 2 2
[149] 2 1 2 1 2 2 2 2 2 1 2 2 2 1 2 2 2 2 2 2 2 2 1 2 2 2 2 1 2 2 2 2 1 2 2 1 1
[186] 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 1 2 2 2 1 2 2 1 1 1 1 2 2 1 1 2 1 1 2
[223] 1 2 2 1 2 2 2 1 2 2 2 2 2 2 2 1 2 1 1 1 1 1 2 2 2 1 1 2 1 1 1 2 1 1 2 1 2
[260] 2 1 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1 2 1 2 2 2 2 1 1 2 1 2 2 1 2 2 2 2 2 2 2
[297] 1 2 2 2 2 1 2 2 2 2 2 1 2 1 2 1 2 2 2 2 2 2 2 2 1 2 1 1 1 1 1 2 2 2 2 2 2
[334] 2 2 2 2 1 1 2 2 1 2 1 1 2 2 2 2 2 1 2 2 2 2 2 2 1 2 2 2 1 2 2 2 2 2 2 2 2
[371] 1 2 2 2 2 2 1 1 2 1 2 1 2 2 2 2 2 2 2 1 1 1 2 2 1 1 1 2 2 2 2 2 1 2 2 1 1
[408] 2 2 2 1 2 2 1 1 1 2 2 2 1 1 2 1 1 2 2 2 2 1 1 1 2 2 2 2 2
attr(,"levels")
[1] "D" "R"

In [74]:
head(Data)

state,district,electedrep,party,medHouseIncome,Bparty
Alabama,1,Bradley Byrne,R,47083,0
Alabama,2,Martha Roby,R,42035,0
Alabama,3,Mike Rogers,R,46544,0
Alabama,4,Robert Aderholt,R,41110,0
Alabama,5,Mo Brooks,R,51690,0
Alabama,6,Gary Palmer,R,61413,0


In [75]:
#supply data to the first model party.bug
Data$logincome<-log(Data$medHouseIncome)
d1 <- list(incomescaled = as.vector(scale(Data$logincome, scale=2*sd(Data$logincome))),
            party = Data$Bparty)

In [76]:
#set dispersed initiate values for model party.bug for 4 chains
inits1 <- list(list(betaincome=10, intercept=10),
               list(betaincome=-10, intercept=10),
               list(betaincome=10, intercept=-10),
               list(betaincome=-10, intercept=-10)
              )

In [44]:
library(rjags)

Loading required package: coda
Linked to JAGS 4.3.0
Loaded modules: basemod,bugs


In [83]:
#run the model with 1000 adaptation
m1 <- jags.model("party.bug", d1, inits1, n.chains=4, n.adapt=1000)

Compiling model graph
   Resolving undeclared variables
   Allocating nodes
Graph information:
   Observed stochastic nodes: 436
   Unobserved stochastic nodes: 438
   Total graph size: 2613

Initializing model



In [84]:
update(m1, 1000)  # burn-in

In [85]:
#sampling from the model and check convergence
x1 <- coda.samples(m1, c("betaincome","intercept"), n.iter=2000)

In [86]:
gelman.diag(x1, autoburnin=FALSE)

Potential scale reduction factors:

           Point est. Upper C.I.
betaincome          1          1
intercept           1          1

Multivariate psrf

1

In [87]:
#sampling from the model after convergence
x1 <- coda.samples(m1, c("betaincome","intercept","prob","partyrep"),
                    n.iter=4000)

In [89]:
#check effective sample size
effectiveSize(x1[,1:4])

betaincome   intercept partyrep[1] partyrep[2] 
   10444.01    10009.03    16000.00    16265.83

In [90]:
#(c)Approximate the posterior mean, posterior standard deviation, and 95% central 
#posterior interval for each parameter.
summary(x1[,1:2])


Iterations = 4001:8000
Thinning interval = 1 
Number of chains = 4 
Sample size per chain = 4000 

1. Empirical mean and standard deviation for each variable,
   plus standard error of the mean:

              Mean     SD  Naive SE Time-series SE
betaincome  0.3588 0.1938 0.0015324      0.0018982
intercept  -0.2147 0.0971 0.0007677      0.0009731

2. Quantiles for each variable:

               2.5%     25%     50%     75%    97.5%
betaincome -0.01728  0.2280  0.3571  0.4927  0.74050
intercept  -0.40412 -0.2809 -0.2146 -0.1491 -0.02608


In [91]:
#(d)Approximate the posterior probability that the “slope” exceeds zero.
mean(as.matrix(x1[,1])>0)

[1] 0.9696875

In [92]:
#(e)Approximate the value of (Plummer’s) DIC and the associated effective number of parameters.
dic.samples(m1,10000)

Mean deviance:  598 
penalty 1.966 
Penalized deviance: 600 

In [93]:
###code for Second Model listing in below

In [94]:
#Create an indexing variable in which the 
#variable state is recoded with the integers 1 to 51.
Data$Bstate<-unclass(Data$state)

In [96]:
#supply data to the first model party2.bug
d2 <- list(incomescaled = as.vector(scale(Data$logincome, scale=2*sd(Data$logincome))),
            party = Data$Bparty,
          state=Data$Bstate)

In [101]:
#set dispersed initiate values for model party2.bug for 4 chains
inits2 <- list(list(betaincome=10, intercept=10,sigmastateinv=1000),
               list(betaincome=-10, intercept=10,sigmastateinv=1000),
               list(betaincome=10, intercept=-10,sigmastateinv=0.1),
               list(betaincome=-10, intercept=-10,sigmastateinv=0.1)
              )

In [111]:
m2 <- jags.model("party2.bug", d2, inits2, n.chains=4, n.adapt=1000)

Compiling model graph
   Resolving undeclared variables
   Allocating nodes
Graph information:
   Observed stochastic nodes: 436
   Unobserved stochastic nodes: 490
   Total graph size: 3112

Initializing model



In [112]:
update(m2, 10000)  # burn-in

In [113]:
#sampling from the model and check convergence
x2 <- coda.samples(m2, c("betaincome","intercept","betastate"), n.iter=2000)

In [114]:
gelman.diag(x2, autoburnin=FALSE)

Potential scale reduction factors:

              Point est. Upper C.I.
betaincome          1.00       1.01
betastate[1]        1.00       1.00
betastate[2]        1.00       1.00
betastate[3]        1.00       1.01
betastate[4]        1.00       1.00
betastate[5]        1.00       1.01
betastate[6]        1.00       1.00
betastate[7]        1.00       1.00
betastate[8]        1.00       1.00
betastate[9]        1.00       1.00
betastate[10]       1.00       1.01
betastate[11]       1.00       1.00
betastate[12]       1.00       1.01
betastate[13]       1.00       1.00
betastate[14]       1.00       1.00
betastate[15]       1.00       1.01
betastate[16]       1.00       1.01
betastate[17]       1.00       1.00
betastate[18]       1.00       1.01
betastate[19]       1.00       1.00
betastate[20]       1.00       1.00
betastate[21]       1.00       1.01
betastate[22]       1.00       1.01
betastate[23]       1.00       1.00
betastate[24]       1.00       1.00
betastate[25]       1.00    

In [128]:
#sampling from the model after convergence
x2 <- coda.samples(m2, c("betaincome","intercept","betastate"), n.iter=8000)

In [129]:
effectiveSize(x2)

betaincome  betastate[1]  betastate[2]  betastate[3]  betastate[4] 
     5531.163     12246.312     15932.550     12219.445      9851.378 
 betastate[5]  betastate[6]  betastate[7]  betastate[8]  betastate[9] 
     3843.568     11880.333      6500.859     14074.513     13751.526 
betastate[10] betastate[11] betastate[12] betastate[13] betastate[14] 
     6547.974     11609.820     10016.638     13577.499      6693.841 
betastate[15] betastate[16] betastate[17] betastate[18] betastate[19] 
    11902.261     16164.393     10127.974     12317.521     11571.533 
betastate[20] betastate[21] betastate[22] betastate[23] betastate[24] 
    17306.216      6655.508      5162.760      9545.439      9627.446 
betastate[25] betastate[26] betastate[27] betastate[28] betastate[29] 
    15144.371     13073.545     16256.170     12644.633     12511.375 
betastate[30] betastate[31] betastate[32] betastate[33] betastate[34] 
     8946.340      7694.785     14950.117      5437.361     10994.366 
betastate[35] betastate[36] betastate[37] betastate[38] betastate[39] 
    16252.496      9686.957      9373.390     10379.707      9103.572 
betastate[40] betastate[41] betastate[42] betastate[43] betastate[44] 
    10182.025     11203.413     17129.263     10599.675      5630.033 
betastate[45] betastate[46] betastate[47] betastate[48] betastate[49] 
    11789.811     13409.926     11228.750      7566.748     10833.605 
betastate[50] betastate[51]     intercept 
    13492.913     13774.807      2204.755

In [130]:
#summary for the “slope” coefficient
summary(x2[,1])


Iterations = 25001:33000
Thinning interval = 1 
Number of chains = 4 
Sample size per chain = 8000 

1. Empirical mean and standard deviation for each variable,
   plus standard error of the mean:

          Mean             SD       Naive SE Time-series SE 
     -0.470867       0.264880       0.001481       0.003580 

2. Quantiles for each variable:

    2.5%      25%      50%      75%    97.5% 
-0.99562 -0.64785 -0.47114 -0.28988  0.04125 


In [148]:
#summary for the “intercept”
summary(x2[,53])


Iterations = 25001:33000
Thinning interval = 1 
Number of chains = 4 
Sample size per chain = 8000 

1. Empirical mean and standard deviation for each variable,
   plus standard error of the mean:

          Mean             SD       Naive SE Time-series SE 
     -0.458587       0.231352       0.001293       0.005030 

2. Quantiles for each variable:

    2.5%      25%      50%      75%    97.5% 
-0.93163 -0.60380 -0.44977 -0.30505 -0.02433 


In [134]:
#(d)Approximate the posterior probability that the “slope” exceeds zero.
mean(as.matrix(x2[,1])>0)

[1] 0.03621875

In [151]:
#(e)largest and smallest posterior mean random effect
a=summary(x2[,2:52])

In [163]:
#(e)largest and smallest posterior mean random effect
which(a$statistics[,1]==max(a$statistics[,1]))
which(a$statistics[,1]==min(a$statistics[,1]))

betastate[22] 
           22

betastate[37] 
           37

In [166]:
#(e)largest and smallest posterior mean random effect
Data[Data$Bstate==22,]

,state,district,electedrep,party,medHouseIncome,Bparty,logincome,Bstate
191,Massachusetts,1,Richard Neal,D,55716,1,10.92802,22
192,Massachusetts,2,Jim McGovern,D,64868,1,11.08011,22
193,Massachusetts,3,Niki Tsongas,D,77995,1,11.26440,22
194,Massachusetts,4,Joseph P. Kennedy III,D,98530,1,11.49812,22
195,Massachusetts,5,Katherine Clark,D,92268,1,11.43245,22
196,Massachusetts,6,Seth Moulton,D,84913,1,11.34938,22
197,Massachusetts,7,Mike Capuano,D,60873,1,11.01655,22
198,Massachusetts,8,Stephen F. Lynch,D,82333,1,11.31853,22
199,Massachusetts,9,Bill Keating,D,68173,1,11.12980,22


In [167]:
#(e)largest and smallest posterior mean random effect
Data[Data$Bstate==37,]

,state,district,electedrep,party,medHouseIncome,Bparty,logincome,Bstate
316,Oklahoma,1,Jim Bridenstine,R,52319,0,10.86511,37
317,Oklahoma,2,Markwayne Mullin,R,40770,0,10.61570,37
318,Oklahoma,3,Frank Lucas,R,47724,0,10.77319,37
319,Oklahoma,4,Tom Cole,R,55183,0,10.91841,37
320,Oklahoma,5,Steve Russell,R,49616,0,10.81207,37


In [168]:
#(f)Approximate the value of (Plummer’s) DIC and the associated effective number of parameters.
dic.samples(m2,10000)

Mean deviance:  510.6 
penalty 31.23 
Penalized deviance: 541.8 